In [4]:
import torch

pc_data = torch.load("./eval-results/surface_reconstruction.pth").detach().cpu().numpy()

In [5]:
pc_data.shape

(400, 6144, 3)

In [7]:
test_data = pc_data[0,...].shape

In [9]:
import open3d as o3d

# Convert to Open3D PointCloud format
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(test_data)

# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Surface reconstruction using Ball Pivoting
radii = [0.005, 0.01, 0.02, 0.04]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

# Visualize the mesh
o3d.visualization.draw_geometries([mesh])


RuntimeError: Unable to cast Python instance of type <class 'int'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)